<a href="https://colab.research.google.com/github/mohammed21kamall/Graduation-Project/blob/main/Graduation-Project/Model%20AI%20/Collabirative%20Filtering/PartOfPredict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
# Define the prediction function using the provided equation
def predict_rating(ru, rv, rvi, Suv, degree_v, n_nearest_users, cu, cv):

    sum_weights = 0
    sum_rating_diff_weighted = 0

    for v in range(1, n_nearest_users + 1):
        sum_rating_diff_weighted += (rvi - rv) * Suv * w(cu, cv)  # Numerator
        sum_weights += (degree_v + 1) * abs(Suv)  # Denominator

    if sum_weights == 0:
        return 0  # Avoid division by zero

    predicted_rating = ru + (sum_rating_diff_weighted / sum_weights)
    return predicted_rating


In [27]:
# Function to get rating for a user and movie
def get_rating(ratings, user_id, movie_id):
    rating = ratings[(ratings['userId'] == user_id) & (ratings['movieId'] == movie_id)]['rating'].values
    if len(rating) > 0:
        return rating[0]
    else:
        return None

In [65]:
# Function to get similar user IDs for a given user ID
def get_similar_user_ids(user_id, Suv, threshold=0.8):
    # Get the similarity row for the given user ID
    similarity_row = Suv.loc[Suv['actual_column_name_for_user_id'] == user_id]
    if similarity_row.empty:
        return "User ID not found!"

    # Sort the similarity values in descending order
    similar_users = similarity_row.drop(columns=['actual_column_name_for_user_id']).sort_values(by=user_id, ascending=False, axis=1)

    # Get the list of similar user IDs
    similar_user_ids = list(similar_users.columns)

    # Remove the user_id from the list if present
    similar_user_ids.remove(user_id) if user_id in similar_user_ids else None

    # Convert similar user IDs to integers and increment by 1
    similar_user_ids = [int(user_id) + 1 for user_id in similar_user_ids]

    return similar_user_ids

The Similarity (Suv)

In [60]:
# Similarity
model_filename = 'drive/MyDrive/Dataset2/SimilarityOfUsers.csv'
Suv = pd.read_csv(model_filename)

Suv = Suv.clip(upper=1.0)
Suv = Suv.applymap(lambda x: max(0, x))
Suv = Suv.applymap(lambda x: x if x >= 0.4 else float(0.0))

Suv.head()

,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
0,1.000000,0.0,0.0,0.516196,0.0,0.0,0.540215,0.0,0.0,0.0,...,0.0,0.480139,0.651735,0.000000,0.436931,0.00000,1.0,1.00000,0.000000,0.410598
1,0.000000,1.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.00000,0.000000,0.000000
2,0.000000,0.0,1.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.00000,0.000000,0.000000
3,0.516196,0.0,0.0,1.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.000000,1.000000,0.000000,0.000000,0.88533,0.0,0.47253,0.000000,0.000000
4,0.000000,0.0,0.0,0.000000,1.0,1.0,0.000000,1.0,0.0,0.0,...,0.0,1.000000,0.000000,0.724984,0.000000,0.00000,0.0,0.00000,0.526469,0.000000


In [69]:
# Prompt the user to enter the user ID
user_id = input("Enter the user ID: ")

# Check if the entered user ID exists in the DataFrame
if user_id not in Suv.index:
    print("User ID not found!")
else:
    # Get similar user IDs for the specified user ID
    similar_user_ids = get_similar_user_ids(user_id, Suv)

    # Display similar user IDs
    print("Similar User IDs:", similar_user_ids)

Enter the user ID: 5
User ID not found!


In [47]:
# Check if the entered user ID exists in the DataFrame
if user_id not in Suv.columns:
    print("User ID not found!")
else:

    # Prompt the user to specify the number of similar user IDs to display
    n = int(input("Enter the number of similar user IDs to display: "))

    # Check if the specified number is greater than the number of similar user IDs found
    if n > len(similar_user_ids):
        print(f"Only {len(similar_user_ids)} similar user IDs found. Displaying all.")

    # Display similar user IDs
    print("User ID:", user_id)
    n_users_from_similar_users = similar_user_ids[:n]
    print(f"Similar User IDs (Top {n}):", n_users_from_similar_users)


Enter the number of similar user IDs to display: 3
User ID: 5
Similar User IDs (Top 3): [4, 5, 7]


In [48]:
n_users_from_similar_users

[4, 5, 7]

In [50]:
# Example: Retrieve the value at row 0, column 0 using column names
Suv_value = Suv.iloc[user_id, similar_user_ids[0]]
print("Suv Value Selected :", Suv_value)



ValueError: Location based indexing can only have [integer, integer slice (START point is INCLUDED, END point is EXCLUDED), listlike of integers, boolean array] types

Rating U (ru) & Rating V (rv)

In [3]:
ratings = pd.read_csv("drive/MyDrive/Dataset2/ratings.csv")
ratings = ratings[['userId', 'movieId', 'rating']]
ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [24]:
# Prompt the user for input for two users and a movie
movie_id = int(input("Enter the movieId: "))
ratings_list = []

for i in range(2):
    user_id = int(input("Enter the userId: "))
    rating = get_rating(ratings, user_id, movie_id)
    ratings_list.append(rating)

# Print all ratings
print("Ratings for movie", movie_id, "are:", ratings_list)

Enter the movieId: 2338
Enter the userId: 1
Enter the userId: 19
Ratings for movie 2338 are: [2.0, 1.0]


In [26]:
ratings_list[1]

1.0